In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
image_folder_path = '/content/drive/MyDrive/Images'


In [ ]:
!pip install opencv-python-headless scikit-image


In [ ]:
import cv2
import numpy as np

def remove_background(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a binary image
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Invert the binary image
    binary = cv2.bitwise_not(binary)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask from the largest contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Remove background
    result = cv2.bitwise_and(image, mask)
    return result


In [ ]:
def denoise_image(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)


In [ ]:
def resample_image(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    return cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)


In [ ]:
def register_image(image, reference_image):
    # Placeholder: Implement actual registration logic here
    return image  # Returning the image as-is for this example


In [ ]:
def normalize_intensity(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)


In [ ]:
def resize_image(image, size):
    return cv2.resize(image, size, interpolation = cv2.INTER_AREA)


In [ ]:
def scale_pixels(image):
    return image / 255.0


In [ ]:
import os
from skimage import io

# Define the path to your dataset
dataset_path = '/content/drive/MyDrive/Images'

# Define the output path
output_path = '/content/drive/MyDrive/processed_dataset'
os.makedirs(output_path, exist_ok=True)

# Define the target size for resizing
target_size = (256, 256)  # Example size

# Process each image in the dataset
for filename in os.listdir(dataset_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other formats if necessary
        img_path = os.path.join(dataset_path, filename)
        image = io.imread(img_path)

        # Apply the processing steps
        image = remove_background(image)
        image = denoise_image(image)
        image = resample_image(image, scale_percent=50)  # Example: resample to 50% of the original size
        image = register_image(image, reference_image=image)  # Using the same image as reference for simplicity
        image = normalize_intensity(image)
        image = resize_image(image, target_size)
        image = scale_pixels(image)

        # Save the processed image
        output_img_path = os.path.join(output_path, filename)
        io.imsave(output_img_path, (image * 255).astype(np.uint8))  # Scaling back to 0-255 and saving


<ipython-input-11-20b01df69078>:31: UserWarning: /content/drive/MyDrive/processed_dataset/2273028514_d7b584f73d.jpg is a low contrast image
  io.imsave(output_img_path, (image * 255).astype(np.uint8))  # Scaling back to 0-255 and saving
